In [2]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame

In [5]:
# Load the csv exported in Part I to a DataFrame
cities_weather = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,62.04,89,99,30.49,AU,1623258100
1,1,karratha,-20.7377,116.8463,74.88,57,98,8.19,AU,1623257834
2,2,barrow,71.2906,-156.7887,35.91,84,71,6.15,US,1623257689
3,3,ushuaia,-54.8000,-68.3000,46.06,49,75,0.00,AR,1623257696
4,4,rikitea,-23.1203,-134.9692,72.68,65,100,15.23,PF,1623257843


# Humidity Heatmap

In [22]:
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

# Configure gmaps
gmaps.configure(api_key = g_key)

# Store latitude and longitude 
locations = cities_weather[["Lat", "Lng"]]

# Store hunidity
humidity = cities_weather["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(30.0, -1.0), zoom_level=2 )

# Add heat layer to map
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [29]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

# Create a DataFrame for cities with good weather conditions
narrow_list = cities_weather[(cities_weather["Max Temp"]>=70) & (cities_weather["Max Temp"]<=80) & (cities_weather["Humidity"]<=50) & (cities_weather["Cloudiness"]==0) & (cities_weather["Wind Speed"]<10)]

# Drop rows with null values
good_weather = narrow_list.dropna()

good_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,naantali,60.4674,22.0243,73.81,46,0,2.42,FI,1623257948
202,202,selma,36.5708,-119.6121,71.04,46,0,6.89,US,1623258137
245,245,casper,42.8666,-106.3131,77.72,25,0,1.99,US,1623258008
318,318,craig,40.5153,-107.5464,77.99,16,0,4.23,US,1623257886
339,339,hamadan,34.7992,48.5146,70.79,14,0,6.91,IR,1623258165
448,448,istanbul,41.0351,28.9833,76.42,47,0,8.34,TR,1623257985


# Hotel Map

In [42]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

# Store into variable named hotel_df
hotel_df = good_weather

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels within 5000 meters
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"

params = {
    "type" : "hotel",
    "keyword" : "hotel",
    "radius" : 5000,
    "key" : g_key
}

# Hit the Google Places API for each city's coordinates
for index, row in hotel_df.iterrows():
    
    # get city name, latitude, and longitude from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Add parameter to params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Assemble URL and make API request
    print(f"Retrieving results for Index {index}: {city_name.capitalize()}.")
    response = requests.get(base_url, params = params).json()
    
    # Extract results
    results = response['results']
    
    # Store the first Hotel result into the DataFrame
    try:
        print(f"Closest hotel in {city_name.capitalize()} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    # If Hotel not available, show No Results found and Store as NaN
    except (KeyError, IndexError):
        print("No Hotel found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = "Nan"
        
    print("----------------")
    
# Print End of search once completed
print("End of Search")
              

Retrieving results for Index 44: Naantali.
Closest hotel in Naantali is Naantali Spa Hotel & Resort.
----------------
Retrieving results for Index 202: Selma.
Closest hotel in Selma is Holiday Inn Selma-Swancourt, an IHG Hotel.
----------------
Retrieving results for Index 245: Casper.
Closest hotel in Casper is Residence Inn by Marriott Casper.
----------------
Retrieving results for Index 318: Craig.
Closest hotel in Craig is Craig Hotel.
----------------
Retrieving results for Index 339: Hamadan.
Closest hotel in Hamadan is hotel 24h.
----------------
Retrieving results for Index 448: Istanbul.
Closest hotel in Istanbul is Radisson Blu Bosphorus Hotel, Istanbul.
----------------
End of Search


In [43]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,44,naantali,60.4674,22.0243,73.81,46,0,2.42,FI,1623257948,Naantali Spa Hotel & Resort
202,202,selma,36.5708,-119.6121,71.04,46,0,6.89,US,1623258137,"Holiday Inn Selma-Swancourt, an IHG Hotel"
245,245,casper,42.8666,-106.3131,77.72,25,0,1.99,US,1623258008,Residence Inn by Marriott Casper
318,318,craig,40.5153,-107.5464,77.99,16,0,4.23,US,1623257886,Craig Hotel
339,339,hamadan,34.7992,48.5146,70.79,14,0,6.91,IR,1623258165,hotel 24h
448,448,istanbul,41.0351,28.9833,76.42,47,0,8.34,TR,1623257985,"Radisson Blu Bosphorus Hotel, Istanbul"


In [44]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))